## this is to run the model with the best regression and RMSE score for the model determined before 

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter


In [2]:
# Load the data
file_path = Path('happiness_joined.csv')
happiness_joined_df = pd.read_csv(file_path,encoding="ISO-8859-1")#,index_col=0)
# happiness_joined_df = df.loc[:, columns].copy()

# df.reset_index(inplace=True, drop=True)

happiness_joined_df.head()


country_code country_name  life_ladder  democracy_index  \
0             8      Albania        4.995             58.9   
1            12      Algeria        4.745             40.1   
2            51      Armenia        5.488             55.4   
3            40      Austria        7.195             82.9   
4            31   Azerbaijan        5.173             27.5   

   consumer_price_index  gender_ratio_males_per100_female  \
0                 119.1                             103.7   
1                 151.4                             102.1   
2                 129.2                              88.8   
3                 118.1                              97.0   
4                 156.9                              99.7   

   infant_mortality_per1000_births  life_expectancy  per_capita_gdp_dollars  \
0                              8.0             78.4                    5303   
1                             21.2             76.6                    3976   
2                             10.8             74.9                    4623   
3                              3.2             81.4                   49701   
4                             20.8             72.8                    4782   

   population_density  safe_drinking_water_access_pct  \
0               105.1                            70.7   
1                18.1                            72.4   
2               103.9                            86.9   
3               108.7                            98.9   
4               121.6                            88.3   

   seats_held_by_women_pct  unemployment_rate  
0                     29.3               12.8  
1                     25.8               11.5  
2                     24.2               16.6  
3                     37.2                4.8  
4                     16.8                6.0

In [3]:
happiness_joined_df[['country_code','democracy_index','consumer_price_index','gender_ratio_males_per100_female','infant_mortality_per1000_births','life_expectancy','per_capita_gdp_dollars','population_density','safe_drinking_water_access_pct','seats_held_by_women_pct','unemployment_rate']] = happiness_joined_df[['country_code','democracy_index','consumer_price_index','gender_ratio_males_per100_female','infant_mortality_per1000_births','life_expectancy','per_capita_gdp_dollars','population_density','safe_drinking_water_access_pct','seats_held_by_women_pct','unemployment_rate']].replace({'\$': '', ',': ''}, regex=True).astype(float)
#
happiness_joined_df.head()
# # REF: https://pbpython.com/currency-cleanup.html

country_code country_name  life_ladder  democracy_index  \
0           8.0      Albania        4.995             58.9   
1          12.0      Algeria        4.745             40.1   
2          51.0      Armenia        5.488             55.4   
3          40.0      Austria        7.195             82.9   
4          31.0   Azerbaijan        5.173             27.5   

   consumer_price_index  gender_ratio_males_per100_female  \
0                 119.1                             103.7   
1                 151.4                             102.1   
2                 129.2                              88.8   
3                 118.1                              97.0   
4                 156.9                              99.7   

   infant_mortality_per1000_births  life_expectancy  per_capita_gdp_dollars  \
0                              8.0             78.4                  5303.0   
1                             21.2             76.6                  3976.0   
2                             10.8             74.9                  4623.0   
3                              3.2             81.4                 49701.0   
4                             20.8             72.8                  4782.0   

   population_density  safe_drinking_water_access_pct  \
0               105.1                            70.7   
1                18.1                            72.4   
2               103.9                            86.9   
3               108.7                            98.9   
4               121.6                            88.3   

   seats_held_by_women_pct  unemployment_rate  
0                     29.3               12.8  
1                     25.8               11.5  
2                     24.2               16.6  
3                     37.2                4.8  
4                     16.8                6.0

In [4]:
happiness_joined_df.columns.values.tolist()

['country_code',
 'country_name',
 'life_ladder',
 'democracy_index',
 'consumer_price_index',
 'gender_ratio_males_per100_female',
 'infant_mortality_per1000_births',
 'life_expectancy',
 'per_capita_gdp_dollars',
 'population_density',
 'safe_drinking_water_access_pct',
 'seats_held_by_women_pct',
 'unemployment_rate']

In [5]:
print(happiness_joined_df.shape)

(91, 13)


## Modeling -- Random Forest regression using all 8 indep variables except "country_name", "country_code",  "life_expectancy" & "consumer_price_index"

In [6]:
# select features 
target = "life_ladder"
features = happiness_joined_df.columns.drop([target,"country_code","country_name"])#, "life_expectancy", "consumer_price_index", "democracy_index", "infant_mortality_per1000_births","gender_ratio_males_per100_female", "population_density"])#,"unemployment_rate"])
X = happiness_joined_df[features]
y = happiness_joined_df[target]
import statsmodels.api as sm
X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

# either 5 variables or 10, but with no splits

                            OLS Regression Results                            
Dep. Variable:            life_ladder   R-squared:                       0.747
Model:                            OLS   Adj. R-squared:                  0.715
Method:                 Least Squares   F-statistic:                     23.60
Date:                Mon, 07 Feb 2022   Prob (F-statistic):           5.95e-20
Time:                        02:49:51   Log-Likelihood:                -71.284
No. Observations:                  91   AIC:                             164.6
Df Residuals:                      80   BIC:                             192.2
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const   

In [24]:
# Normal train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
# Resample the training data with the BalancedRandomForestRegression
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=78) 
# instantiate model
model = RandomForestRegressor()
# ref: https://machinelearningmastery.com/random-forest-ensemble-in-python/
# fit model
model.fit(X, y)
# fit
model.fit(X_train, y_train)
# predict
y_pred = model.predict(X_test)
from sklearn.preprocessing import StandardScaler
# 3) Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# 4) Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances
print("X_train_mean:", np.mean(X_train_scaled[:,0]))
print("X_test_mean:", np.mean(X_test_scaled[:,0]))
print("X_train_std:", np.std(X_train_scaled[:,0]))
print("X_test_std:", np.std(X_test_scaled[:,0]))
# model evaluation
# ref: https://towardsdatascience.com/multiple-regression-as-a-machine-learning-algorithm-a98a6b9f307b
from sklearn.metrics import r2_score, mean_squared_error
score = r2_score(y_test, y_pred)
print("Random Forest R2score:", score)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Random Forest root mean squared error: ', rmse)
# ref: https://datagy.io/python-sklearn-linear-regression/

X_train_mean: 4.195989960715665e-16
X_test_mean: -0.3231427339702866
X_train_std: 1.0
X_test_std: 0.9622616262978819
Random Forest R2score: 0.36939771191899917
Random Forest root mean squared error:  0.7424768027469888


In [ ]:
# Make predictions using the test data
y_pred = regressor.predict(X)
results = pd.DataFrame({
    "Prediction": y_pred, 
    "Actual": y
}).reset_index(drop=True)
results.head()

In [20]:
# The slope
# The y-intercept
print(regressor.coef_)
print(regressor.intercept_)

AttributeError: 'RandomForestRegressor' object has no attribute 'coef_'

In [21]:
# summary_df that shows the line equation of the model: 
# initialize list of lists
data = [['MultiVariable Linear Regression', score, rmse, model.intercept_, 7.65906048e-03, 9.81913381e-03, -8.44061880e-03, 1.48696587e-05, -8.73823382e-05, 1.03343798e-02, 1.32232819e-02, -3.63415910e-02]]
 
# Create the pandas DataFrame
summary_df = pd.DataFrame(data, columns = ['Name of model:', 'R-squared_score', 'RMSE_Score', 'intercept', 'coeff1', 'coeff2', 'coeff3' ,'coeff4', 'coeff5','coeff6', 'coeff7', 'coeff8' ])

summary_df.round(5)

AttributeError: 'RandomForestRegressor' object has no attribute 'intercept_'

In [ ]:
# https://towardsdatascience.com/super-simple-machine-learning-by-me-multiple-linear-regression-part-1-447800e8b624
# https://www.datacourses.com/find-p-value-significance-in-scikit-learn-3810/
# https://www.w3schools.com/python/python_ml_multiple_regression.asp

# https://www.quora.com/What-are-some-good-machine-learning-algorithms-for-predicting-a-continuous-variable-with-N-number-of-inputs
# https://www.jigsawacademy.com/popular-regression-algorithms-ml/   (imp)


# https://colab.research.google.com/drive/1TY5M27edj_Aeh-vWE42SUMUwv49w-EW5?usp=sharing&authuser=1#scrollTo=Efpl-zrCO27f

In [ ]:
# rerun the significant value >0.05 value for the indep cvariables
# seperate with subheadings for each variables 

In [ ]:
# select features 
#X = happiness_joined_df[['democracy_index','consumer_price_index','gender_ratio_males_per100_female','infant_mortality_per1000_births','life_expectancy','per_capita_gdp_dollars','population_density','safe_drinking_water_access_pct','seats_held_by_women_pct','unemployment_rate']]
#y = happiness_joined_df["life_ladder"]
# ref: https://www.pluralsight.com/guides/importing-and-splitting-data-into-dependent-and-independent-features-for-ml
# ref: https://towardsdatascience.com/multiple-regression-as-a-machine-learning-algorithm-a98a6b9f307b


target = "life_ladder"
features = happiness_joined_df.columns.drop([target,"country_code","country_name", "life_expectancy", "consumer_price_index", "democracy_index", "infant_mortality_per1000_births","gender_ratio_males_per100_female", "population_density"])#,"unemployment_rate"])
X = happiness_joined_df[features]
y = happiness_joined_df[target]